In [283]:
%reset -f

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Polar Json converter

Last updated by AKR (8/13/24).

This notebook converts "manually exported" Polar json files into csv files.

**Format of csv file output:**

**f"{participant id}\_{polar workout number (pwo)}\_{day since start date}_hr_WearablePolar_Polar.csv"**

For example: **019_pwo04_d006_hr_WearablePolar_Polar.csv** is participant 19's 4th workout, completed 6 days after the study start date.

**Details:**
- **pwo (polar workout number)**: workouts are counted if they are >5 minutes in duration. After passing this filter, they are numbered sequentially by date.
- **d (days)**: workouts are named in reference to the "study start date," defined as the the first recorded date in the Week 1 Workout Log in REDCap. This annotation in REDCap is the "ground truth" for when participants began exercising. Oftentimes, they test the device before the study or complete a workout session at the CTRU before the intervention. This will appear as negative days (e.g. 4 days before study is -d004). In other words, the first Polar entry may be a "test" or CTRU workout, so REDCap is the best and closest annotation to training start date.


Each csv file represents an individual workout, with columns for dateTime and (HR) values. 

We expect participants to have 36 workouts (pwo01 - pwo36) if they have completed 3 workouts per week for 12 weeks. On average, participants would be expected to have participated for ~84 days in the study (i.e. 12 weeks), but they are allowed 2 week extensions for non-adherence, illness, or to accomodate for scheduling. In rare cases of injury, participants were extended beyond 14 weeks.



### Load packages

In [1]:
import random
import os, glob
import pandas as pd
from datetime import datetime, timedelta
import json 
from IPython.display import display, HTML
import matplotlib.pyplot as plt

#initialization
random.seed(1000)

In [33]:
import os
import re

json_path = "/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar_recovery/"

# List only directories (folders)
folders = [f for f in os.listdir(json_path) if os.path.isdir(os.path.join(json_path, f))]

# Extract leading numbers and sort accordingly
folders_sorted = sorted(folders, key=lambda x: int(re.match(r"(\d+)", x).group()) if re.match(r"(\d+)", x) else float('inf'))

folders_sorted

['003_qtz1b14715581577341838',
 '004_qtz1b18445579426718188',
 '005_qtz1b16767772887265127',
 '006_qtz1b13893369732763681',
 '010_qtz1b19121547781291613',
 '012_qtz1b17269433695528197',
 '014_qtz1b13168362397931536',
 '018_qtz1b19261322566215748',
 '019_qtz1b17236523485537658',
 '020_qtz1b19129928133897239',
 '021_qtz1b16327775253853113',
 '030_qtz1b17855938635599315',
 '036_qtz1b18341388618579643',
 '037_qtz1b19851346611142986',
 '040_qtz1b12182259692885175',
 '045_qtz1b14361342418376156',
 '046_qtz1b12637499885254663',
 '047_qtz1b11649673216856592',
 '051_qtz1b16917131625167555',
 '060_qtz1b13779232687428538',
 '064_qtz1b12351664185336321',
 '071_qtz1b19649447249479579',
 '072_qtz1b19779165843537516',
 '073_qtz1b17592513599164541',
 '075_qtz1b19516699726434684',
 '077_qtz1b11121669136412997',
 '080_qtz1b19331948298967664',
 '084_qtz1b11871696423624282',
 '086_qtz1b11984351926876373',
 '087_qtz1b19955871415948163',
 '089_qtz1b15536926332385566',
 '090_qtz1b15741679182868673',
 '092_qt

In [37]:
# Figure out folders that need to be processed

folder1 = '/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar_recovery/'
folder2 = '/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/'

# Get folder names in each directory
folders1 = set(f for f in os.listdir(folder1) if os.path.isdir(os.path.join(folder1, f)))
folders2 = set(f for f in os.listdir(folder2) if os.path.isdir(os.path.join(folder2, f)))

# Get folder names in folder2 not in folder1
unique_to_folder2 = folders1 - folders2

print("Folders in raw (aka polar recovery) but not in processed (aka polar workout):")
for folder in unique_to_folder2:
    print(folder)

Folders in raw (aka polar recovery) but not in processed (aka polar workout):
hr
pl_rec_drop_075
087_qtz1b19955871415948163
040_qtz1b12182259692885175
1076_qtz1b12335429866786569


### Function to gets files for a participant

In [36]:
def get_files(participant):
    json_path = f"/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar_recovery/{participant}/polar-user-data-export_{participant}/"
    #json_path = f"/Users/aubreykr/Library/CloudStorage/GoogleDrive-aubreykr@stanford.edu/Shared drives/HIIT and Endurance Study/Data/data/polar_recovery/097_qtz1b15382499214278169/polar-user-data-export_097_qtz1b15382499214278169/"
    #json_path = f"/Users/aubreykr/Library/CloudStorage/GoogleDrive-aubreykr@stanford.edu/Shared drives/HIIT and Endurance Study/Data/data/polar_recovery/060_qtz1b13779232687428538/polar-user-data-export_060_qtz1b13779232687428538/"
    os.chdir(json_path)
    files = []
    for file in glob.glob("training-session*.json"):
        files.append(file)
    original_files = files.copy()
    files = sorted(files)
    print("There are", len(files), "files for", participant)
    return files
    
#get_files("060_qtz1b13779232687428538")

get_files("133_qtz1b18356435748111734")

convert_polar("133_qtz1b18356435748111734")

There are 50 files for 133_qtz1b18356435748111734
There are 50 files for 133_qtz1b18356435748111734
CSV file saved: /Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/133_qtz1b18356435748111734/133_pwo01_d0-5_hr_WearablePolar_Polar.csv
CSV file saved: /Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/133_qtz1b18356435748111734/133_pwo02_d000_hr_WearablePolar_Polar.csv
CSV file saved: /Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/133_qtz1b18356435748111734/133_pwo03_d001_hr_WearablePolar_Polar.csv
CSV file saved: /Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/133_qtz1b18356435748111734/133_pwo04_d002_hr_WearablePolar_Polar.csv
CSV file saved: /Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/133_qtz1b18356435748111734/133_pwo05_d003_hr_WearablePolar_Polar.csv
CSV file saved:

### Function to convert Polar json to csv

In [4]:
def convert_polar(participant):

    # 1) REDCAP DATA
    
    # Load participant workout logs from REDCap (goal: obtain workout dates from handwritten logs to compare with Polar wearable data).
    notes_path = '/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/Redcap/Raw_Data/64208_WorkoutTimeStamps_Raw.csv'
    df_notes_orig = pd.read_csv(f"{notes_path}")
    df_notes= df_notes_orig.copy()

    # Convert all workout dates to datetime
    df_notes['workout_1'] = pd.to_datetime(df_notes['workout_1']) 
    df_notes['workout_2'] = pd.to_datetime(df_notes['workout_2'])
    df_notes['workout_3'] = pd.to_datetime(df_notes['workout_3'])
    df_notes['workout1_date_only'] = df_notes['workout_1'].dt.strftime("%Y-%m-%d")
    df_notes['workout2_date_only'] = df_notes['workout_2'].dt.strftime("%Y-%m-%d")
    df_notes['workout3_date_only'] = df_notes['workout_3'].dt.strftime("%Y-%m-%d")


    # 2) Get POLAR from Google Drive
    
    json_path = f"/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar_recovery/{participant}/polar-user-data-export_{participant}/"
    os.chdir(json_path)
    files = []
    for file in glob.glob("training-session*.json"):
        files.append(file)
    original_files = files.copy()
    files = sorted(files)
    print("There are", len(files), "files for", participant)

    
    # For each each Polar json file, extract HR data; then save as a csv file in Google Drive.
    # Create loop to iterate through Polar json files
    i=0
    counter=0
    for i, file in enumerate(files):

        # Get full path to file
        full_file_name = f"{json_path}{file}"

        # Read json data from the file and store it as a string
        with open(full_file_name, 'r', ) as file2read:
            json_data = file2read.read()
            file2read.close()

        # Parse json into a data dictionary with json.loads()
        data_dict = json.loads(json_data)

        # Extract key of interest
        exercises = data_dict['exercises']

        # Get data from Polar file (e.g. extract values, convert to dataframe)
        try:
            df_hr = pd.DataFrame(exercises[0]['samples']['heartRate'])

        # Print if there is an error but keep running
        except (KeyError, IndexError) as e:
            # Handle the case where 'exercises' is missing or empty
            print(f"The file {participant} might be empty! Error: {e}")
            continue
        
        # Convert time column to datetime type
        df_hr['dateTime'] = pd.to_datetime(df_hr['dateTime'])

    # 3) Enrich Polar file with REDCap workout annotations
        
        # a) Extract date of each workout as a string
        date_info = df_hr['dateTime'].iloc[0].strftime("%Y-%m-%d")
        workout_date = pd.to_datetime(date_info).date() # Convert string to date

        # b) Get redcap start date for each ppt
        participant_num = int(participant.split('_')[0]) # Get ppt number from polar file
        start_date_str = df_notes.loc[(df_notes['redcap_event_name'] == 'week_1_arm_1') & (df_notes['record_id'] == participant_num), 'workout_1'].values[0]
        start_date = pd.to_datetime(start_date_str).date() # Convert string to date

        # c) Compute day of workout (d) relative to redcap start date 
        delta = workout_date - start_date
        days_str = str(delta.days)
        day = f'd{days_str:0>3}'

        # d) If the workout is longer than 5 minutes, label it as a polar workout (pwo)
        if df_hr['dateTime'].max() - df_hr['dateTime'].min() > timedelta(minutes=5):

            # Update pwo number
            counter += 1
            workout_num = f'pwo{counter:02}'


        # e) Rename file in desired format with participant ID, polar workout number (pwo), and day (d) since study start
            wo_info = f'{workout_num}_{day}'
            output_path = f"/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/{participant}/"
            output_name = f"{participant.split('_')[0]}_{wo_info}_hr_WearablePolar_Polar.csv" 
            fullname = os.path.join(output_path, output_name)   


        # f) Save to drive
            if not os.path.exists(output_path):  
                os.mkdir(output_path) 

            if not os.path.exists(fullname): 
                df_hr.to_csv(fullname, index=False)
                print(f"CSV file saved: {fullname}")
            
            # If files already exist, don't overwrite!
            else:
                print(f"File already exists and will not be overwritten.") 

    print("Processing complete.")
    print(f'Participant {participant_num}: {counter} workouts logged.')
    

In [21]:
convert_polar("003_qtz1b14715581577341838")

There are 27 files for 003_qtz1b14715581577341838
CSV file saved: /Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/003_qtz1b14715581577341838/003_pwo01_d000_hr_WearablePolar_Polar.csv
CSV file saved: /Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/003_qtz1b14715581577341838/003_pwo02_d001_hr_WearablePolar_Polar.csv
CSV file saved: /Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/003_qtz1b14715581577341838/003_pwo03_d003_hr_WearablePolar_Polar.csv
CSV file saved: /Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/003_qtz1b14715581577341838/003_pwo04_d007_hr_WearablePolar_Polar.csv
CSV file saved: /Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/003_qtz1b14715581577341838/003_pwo05_d008_hr_WearablePolar_Polar.csv
CSV file saved: /Users/aubreykr/Google Drive/Shared drives/HIIT a

## Plot the QC check files

In [7]:
# Plot the QC check files
participant = "073_qtz1b17592513599164541"
base_dir = f"/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/polar/workout/{participant}/"

# Load redcap data
df_path = '/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/Redcap/Raw_Data/64208_HRZones_Raw.csv'
df_red = pd.read_csv(df_path)

ppt= participant.split('_')[1]
participant_info = df_red[df_red['myphd_id'] == ppt]['record_id'].iloc[0]

# Pattern to match QC_CHECK files
pattern = os.path.join(base_dir, "*.csv")

# Get a list of files that match the pattern
qc_check_files = glob.glob(pattern)

# Print the list of QC_CHECK files
duration=0
for file in qc_check_files:
    df = pd.read_csv(file)

    # Convert 'dateTime' column to datetime objects
    df['dateTime'] = pd.to_datetime(df['dateTime'])

    # Get duration
    duration = df['dateTime'].max() - df['dateTime'].min()

    # Get group
    rand_group = df_red[df_red['myphd_id'] == ppt]['randomization_group'].values[0]

    # Get HR zones
    MICT_lower_val = df_red[df_red['myphd_id'] == ppt]['target_hr_45'].values[0]
    MICT_upper_val = df_red[df_red['myphd_id'] == ppt]['target_hr_55'].values[0]
    HIIT_lower_val = df_red[df_red['myphd_id'] == ppt]['target_hr_70'].values[0]

    # Extract the time in HH:MM format
    df['dateTime'] = df['dateTime'].dt.strftime('%H:%M')

    # Plot the data
    plt.figure(figsize=(12,6))
    plt.plot(df['dateTime'], df['value'], label='Heart Rate')
    plt.xlabel('Time (HH:MM)')
    plt.ylabel('Heart Rate (bpm)')
    plt.title(f'{duration}, {file.split("/")[11]}')
    plt.axhline(y=MICT_lower_val, color='g', linestyle='--')
    plt.axhline(y=MICT_upper_val, color='y', linestyle='--')
    plt.axhline(y=HIIT_lower_val, color='r', linestyle='--')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()